### In this notebook I want to explain how to work the binary classifier. First I want to show the Perceptron diagram. 


In [11]:
# import libraries.
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

array([[4, 5, 6],
       [7, 8, 9]])